### Q2 四种模型的表现回测，以及调仓频率对模型预测效果的影响研究

Q2通过对不同模型的回测，判断最优的调仓频率，并比较模型之间的预测效果。

#### 1. 研究模型
- 均值方差模型 （MVO）
- Black-litterman模型 （BL）
- 风险平价模型 （RP）
- 风险预算模型 （RB）

模型具体实现：`model/`，测试覆盖了以90/150/210/270天的四个调仓频率。

由于BL模型的计算过程中需要输入投资者对资产的观点，在此实验中BL模型选择使用真实数据（转换成对角矩阵，每一行代表单一资产的实际变化情况）。详情请见：[model/bl_model.py](model/bl_model.py)。

由于RB模型的计算过程中需要提供风险预算分配情况，此实验分别使用了三种方法：（1）使用资产未来实际收益情况，记作'RB'；（2）使用资产历史收益情况，记作'RB-H'；（3）使用garch模型，以资产历史波动来计算未来波动情况，记作'RB-G'。详情请见：[model/risk_budget_model.py](model/risk_budget_model.py)。

#### 2. 测试数据

本实验使用A股每日收盘价进行计算，文件位置：[data/aidx_eod_prices.csv](data/aidx_eod_prices.csv)

实验过程中随机挑选100种资产组合（数量在5-10只，相关系数小于50%）进行计算。

#### 3. 分析结果概况
- MVO，BL模型的波动太大。RP模型表现出稳定低波动和负收益（与平权比较）。
- RB模型表现出稳定低波动和正收益（与平权比较）。
- 调仓频率对模型预测表现影响不大，不同调仓频率之间结果在同一数量级。
- 对于RB模型，使用未来实际数据预测效果最好。garch模型比直接使用资产历史收益情况表现更好。

#### 4. 注意事项
- Notebook无法保存可互动图片，需要重新运行程序才能获得可互动结果。互动图中的图例是可点击，用于显示和隐藏对应的线。
- 此实验需要14分钟左右时间运行（4个模型，4个调仓频率，100个随机资产组合）。
- 在此实验数据中，很难找到20只相关系数小于70%的资产组合，所以实验选取5-10只作为一组。

In [ ]:
"""
Package Import
"""

import tool # custom tool for model calling

import scipy, random
import pandas as pd
import numpy as np
from collections import defaultdict

%matplotlib widget
import matplotlib.pyplot as plt
from ipywidgets import interact, fixed, IntSlider, IntText

import warnings
warnings.filterwarnings('ignore')

In [ ]:
"""
Data import
"""

asset_index = pd.read_csv("data/aidx_eod_prices.csv")

# data sorting, longer than 800 days
grouped_asset = asset_index.groupby("S_IRDCODE")
asset_dfs = {ird_code: group for ird_code, group in grouped_asset if len(group) >= 800}
for ird_code, grouped_df in asset_dfs.items():
    grouped_df['TRADE_DT'] = pd.to_datetime(grouped_df['TRADE_DT'], format='%Y%m%d')
    grouped_df.sort_values(by='TRADE_DT', inplace=True)

#### 参数输入

下方代码包含三类可调参数：

- 模型所需参数：目标回报，无风险率
- 实验运行设置：随机次数，资产组合数量限制，组合内资产相关性限制
- 实验测试对象：调仓频率，待测试模型

In [ ]:
"""
Parameters
"""

BACKTEST_DAY = 30 # lookback period (not used)
TARGET_RETURN = 0.0 # target return
RISK_FREE_RATE = 0.02 # risk-free rate

NUM_ITERATION = 100 # test amounts
NUM_LIMIT = (5,10) # assets amount limitation range
CORR_LIMIT = 0.5 # assets correlation limiation

REBALANCE_DAYS = [90, 150, 210, 270] # rebalancing days for test
MODEL_TYPES = ['MVO', 'BL', 'RP', 'RB'] # test models 1
# MODEL_TYPES = ['RB', 'RB-H', 'RB-G'] # test models 2

In [ ]:
"""
Model rebalancing function
"""

def rebalance(asset_index, rebalance_day, weight_constraints, model_type):
    predicts = [] # the predict result from model
    actuals = [] # the actual performance of model
    realities = [] # the actual performance with equal-weight
    
    for i in range(rebalance_day, len(asset_index), rebalance_day):
        
        if i+rebalance_day >= len(asset_index):
            break
        
        historical_data = asset_index[i-rebalance_day:i]
        future_data = asset_index[i:i+rebalance_day]
        
        predict, actual = tool.evaluate(historical_data, future_data, weight_constraints, model_type, TARGET_RETURN, RISK_FREE_RATE)
        predicts.append(predict)
        actuals.append(actual)
        
        # equally weighed
        reality = tool.check([1 / len(asset_index.columns) for _ in range(len(asset_index.columns))], future_data, RISK_FREE_RATE)
        realities.append(reality)
    
    return predicts, actuals, realities

In [ ]:
"""
Asset sampling
"""

def sample(num_limit, asset_dfs, corr_limit):
    index_list = random.sample(list(asset_dfs.keys()), num_limit)
    
    def is_non_related(index_list):
        for i in range(0, len(index_list)):
            for j in range(i+1, len(index_list)):
                i_df = asset_dfs[index_list[i]]
                j_df = asset_dfs[index_list[j]]
                min_length = min(len(i_df['PCHG']), len(j_df['PCHG']))
                corr, _ = scipy.stats.spearmanr(i_df['PCHG'].iloc[:min_length], j_df['PCHG'].iloc[:min_length])
                if corr > corr_limit:
                    return False
        return True
    
    while is_non_related(index_list) == False:
        index_list = random.sample(list(asset_dfs.keys()), num_limit)
    
    return index_list

# sample(20, asset_dfs, 0.3)

In [ ]:
"""
Different Models with the same assets (randomly generated) and different rebalancing days
"""

def asset_rebalance(asset, num_limit, model_types, rebalancing_days, asset_dfs, corr_limit):
    
    asset_index = asset.copy()
    
    # randomly select assets
    actual_num_limit = np.random.randint(*num_limit)
    index_list = sample(actual_num_limit, asset_dfs, corr_limit)
    asset_index['TRADE_DT'] = pd.to_datetime(asset_index['TRADE_DT'], format='%Y%m%d')
    asset_index.sort_values(by='TRADE_DT', inplace=True)
    asset_index.set_index('TRADE_DT', inplace=True)
    asset_index = asset_index.pivot(columns='S_IRDCODE', values='CLOSE').ffill()[index_list].dropna()
    
    # weight constraints
    n = len(index_list)
    index_min_weight = [0 for _ in range(n)]
    index_max_weight = [1 for _ in range(n)]
    weight_constraints = list(zip(index_min_weight, index_max_weight))
    
    # start iteration
    results = {}
    for model_type in model_types:
        for rebalance_day in rebalancing_days:
            _, actuals, realities = rebalance(asset_index, rebalance_day, weight_constraints, model_type)
            results[(model_type, rebalance_day)] = list(zip(*actuals))
            results[('EW', rebalance_day)] = list(zip(*realities))
    
    return results, index_list

# results, index_list = asset_rebalance(asset_index, (10,11), MODEL_TYPES, [180, 210], asset_dfs, CORR_LIMIT)
# results = dict(sorted(results.items(), key=lambda item: item[0][1]))
# print(index_list)
# print(results)

In [ ]:
"""
Calculate the average for one assets combination with each (model, rebalance_day)
"""

def calculate_averages(data, exclude_model='EW'):
    grouped_data = defaultdict(dict)

    # Group data by the second key of the tuple
    for (model, period), values in data.items():
        grouped_data[period][model] = values

    results = {}
    draws = defaultdict(dict)

    # Perform division and calculate averages
    for period, models in grouped_data.items():
        ew_values = models.get(exclude_model)
        if ew_values is None:
            continue  # Skip if 'EW' data is not present

        for model, values in models.items():
            if model != exclude_model:
                modified_values = []
                for index, (value, ew_value) in enumerate(zip(values, ew_values)):
                    if index == 0:  # For the first set, use subtraction (return)
                        result = [v - ew for v, ew in zip(value, ew_value)]
                    else:  # For the other sets, use division (volatility & sharpe)
                        result = [v / ew if ew != 0 else 0 for v, ew in zip(value, ew_value)]
                    modified_values.append(result)
                    
                averages = [sum(value) / len(value) for value in modified_values]
                results[(model, period)] = averages
                
                for i, value_set in enumerate(modified_values):
                    draws[i][(model, period)] = value_set

    return results, draws

# t = calculate_averages(results)
# print(t)

#### 实验开始

在实验中的每次循环下，实验会根据相关性限制和资产数量限制来随机挑选资产组合。在一次完整循环中，资产组合对于的所有（模型，调仓频率）预测情况将被计算，通过上方`asset_rebalance()`函数。

实验过程中会计算比较每个模型+调仓频率的表现（模型预测生成的权重vs平权），通过三个指标：

- 回报：模型预测权重（T之前）x 实际收益情况（T之后）- 平权 x 实际收益情况（T之后）
- 波动：模型预测权重对应的实际收益波动 / 平权对应的实际收益波动
- 夏普：模型预测权重对应的夏普率 / 平权对应的夏普率

注意事项：
- T代表调仓时刻，即模型预测发生时刻，T之前代表历史收益数据，T之后代表未来实际收益情况。
- 统计过程查看[tool.py](tool.py)中[check()](tool.py#L49)函数，以及上方`calculate_averages()`函数。
- 实验过程中模型预测失败情况下，则会用平权代替模型计算权重。

In [ ]:
"""
Iteration start
"""

final_results = []
final_draws = []
index_lists = []
for i in range(0, NUM_ITERATION):
    results, index_list = asset_rebalance(asset_index, NUM_LIMIT, MODEL_TYPES, REBALANCE_DAYS, asset_dfs, CORR_LIMIT)
    results = dict(sorted(results.items(), key=lambda item: item[0][1]))
    results, draws = calculate_averages(results)
    final_results.append(results)
    final_draws.append(draws)
    index_lists.append(index_list)

print(final_results)

#### 资产调仓周期显示

下方代码提供可互动图来展示每组资产在对应的（模型，调仓频率）下，完整的一次调仓周期结果。

注意事项：
- X轴为每次调仓操作，Y轴为模型和平权的比较结果数值，图中上方显示资产代码信息。
- 图表左上方可输入index来检视不同资产组合调仓情况，图例可点击来显示/隐藏图线。


<img src="img/调仓周期示例.png" width="60%" />

In [ ]:
"""
Display a complete adjustment period under different (model, period) for a single 
"""

def display_draws(final_draws, index_lists, idx):
    print(index_lists[idx])
    draws = final_draws[idx]
    num_plots = len(draws)
    plot_width = max(6, num_plots * 4)  # Adjust width dynamically based on number of plots
    fig, axes = plt.subplots(1, num_plots, figsize=(plot_width, 4))
    titles = ['Return', 'Volatility', 'Sharpe Ratio']
    
    # If there's only one plot, make axes iterable
    if num_plots == 1:
        axes = [axes]
    
    line_groups = {}
    line_to_label_map = {}

    # Plotting
    for i, (ax, title) in enumerate(zip(axes, titles)):
        for (model, period), value_set in draws[i].items():
            line, = ax.plot(value_set)
            label = f'{model}, {period}'
            if label not in line_groups:
                line_groups[label] = []
            line_groups[label].append(line)
        ax.set_title(f'{title}')
    
    legend = fig.legend([list(group)[0] for group in line_groups.values()], line_groups.keys(), loc='lower center', bbox_to_anchor=(0.5, -0.01), ncol=8)

    # Update line_to_label_map
    for leg_line, text in zip(legend.get_lines(), legend.get_texts()):
        line_to_label_map[leg_line] = text.get_text()

    # Make legend clickable
    def on_legend_click(event):
        leg_line = event.artist
        label = line_to_label_map.get(leg_line)
        if label:
            lines = line_groups[label]
            visible = not lines[0].get_visible()  # Toggle based on the first line's visibility
            for line in lines:
                line.set_visible(visible)
            leg_line.set_alpha(1.0 if visible else 0.2)
        fig.canvas.draw()

    for leg_line in legend.get_lines():
        leg_line.set_picker(5)  # 5 pts tolerance
    
    fig.canvas.mpl_connect('pick_event', on_legend_click)

    plt.tight_layout()
    plt.subplots_adjust(bottom=0.2)
    plt.show()


interact(display_draws, 
         final_draws=fixed(final_draws), 
         index_lists=fixed(index_lists), 
         idx=IntText(value=0, description='Index:', min=0, max=len(final_draws)-1))

In [ ]:
"""
Output (for each optimization)
"""
def aggregate_results(dicts):
    aggregated_results = {}

    # Initialize aggregated_results with empty lists for each key
    for key in dicts[0].keys():
        aggregated_results[key] = []

    # Iterate over each dictionary
    for d in dicts:
        for key, values in d.items():
            # Assuming all dictionaries have the same structure
            for i, value in enumerate(values):
                if len(aggregated_results[key]) <= i:
                    aggregated_results[key].append([])
                aggregated_results[key][i].append(value)

    # Convert lists of values to tuples
    for key in aggregated_results:
        aggregated_results[key] = [tuple(lst) for lst in aggregated_results[key]]

    return aggregated_results

aggregated_results = aggregate_results(final_results)
print(aggregated_results)

#### 实验结果显示

下方代码将完整的100次随机资产组合测试结果呈现出来。下图将完整的一个调仓周期数据计算平均，统计为一点。

注意事项：
- X轴代表不同资产（即NUM_ITERATIONS）。Y轴为对应资产组合下模型权重与平权比较的完整调仓周期的平均值。
- 图标上方滚动轴用于调整不同比较指标，图例可点击用于显示/隐藏图线。

<div style='text-align: left;'>
    <img src="img/实验结果示例.png" width="55%" />
</div>

In [ ]:
"""
Visualisation (for each optimization)
"""

def asset_display(data, i=0):
    line_styles = ['-', '--', ':']
    colors = plt.cm.viridis(np.linspace(0, 1, len(data)))

    line_style = line_styles[i]

    fig, ax = plt.subplots(figsize=(10, 6))
    lines = []

    for (key, lists), color in zip(data.items(), colors):
        if i < len(lists):
            lst = lists[i]
            l = np.array(lst)
            l = l[~np.isnan(l)]
            line, = ax.plot(l, line_style, color=color, label=f'{key}')
            lines.append(line)
    
    if i == 0:
        ax.axhline(y=0, color='red', linestyle=':')
    elif i == 1:
        ax.axhline(y=1, color='red', linestyle=':')
        
    plt.subplots_adjust(right=0.7)
    leg = ax.legend(fancybox=True, shadow=True, loc='center left', bbox_to_anchor=(1, 0.5), ncol=2)
    
    lined = {}
    for legline, origline in zip(leg.get_lines(), lines):
        legline.set_picker(5)
        lined[legline] = origline
        
    for legline, line in zip(leg.get_lines(), lines):
        legline.set_alpha(0.2)
        line.set_visible(False)

    avg_text_objects = {}
    
    def on_pick(event):
        legline = event.artist
        origline = lined[legline]
        visible = not origline.get_visible()
        origline.set_visible(visible)
        
        if visible:
            if origline not in avg_text_objects:
                display_position = (0.05, 0.95 - 0.05 * len(avg_text_objects))
                avg_value = np.nanmean(origline.get_ydata())
                avg_text_objects[origline] = ax.text(display_position[0], display_position[1], 
                                                     f'Avg {legline.get_label()}: {avg_value:.4f}', 
                                                     transform=ax.transAxes, color=origline.get_color(),
                                                     fontsize=9, verticalalignment='top')
            else:
                avg_text_objects[origline].set_visible(True)
        else:
            if origline in avg_text_objects:
                avg_text_objects[origline].set_visible(False)
                del avg_text_objects[origline]

        legline.set_alpha(1.0 if visible else 0.2)
        fig.canvas.draw()

    fig.canvas.mpl_connect('pick_event', on_pick)

    titles = {0 : 'Return', 1 : 'Volatility', 2 : 'Sharpe Ratio'}
    ax.set_title(titles.get(i, 'Q2 Line Plots'))
    ax.set_xlabel('Test Index')
    ax.set_ylabel('Value')

    plt.show()


interact(asset_display,
         data=fixed(aggregated_results),
         i=IntSlider(min=0, max=2, step=1, value=0))

### Q2 实验结果分析

#### 1. 调仓频率和模型关系：

风险均值模型：风险均值模型在任何调仓频率下，波动幅度都非常大。在本次回测结果中，无法显示出调仓频率对均值方差模型的影响。

<div style="text-align: left;">
    <img src="img/mvo_return.png" style="width: 45%; margin-right: 5px;" />
    <img src="img/mvo_vol.png" style="width: 45%;" />
</div>

Black-litterman模型：BL模型在任何调仓频率下，波动幅度都非常大。在本次回测结果中，无法显示出调仓频率对BL模型的影响。

<div style="text-align: left;">
    <img src="img/bl_return.png" style="width: 45%; margin-right: 5px;" />
    <img src="img/bl_vol.png" style="width: 45%;" />
</div>

风险平价模型：风险平价模型在不同调仓频率下均保持比较稳定的情况。在下图中，可以发现对于风险平价模型而言，回报越高，对应的波动情况也越高。以此次结果而言，以270天作为调仓频率，风险平价模型有最好的预测表现。

<div style="text-align: left;">
    <img src="img/rp_return.png" style="width: 45%; margin-right: 5px;" />
    <img src="img/rp_vol.png" style="width: 45%;" />
</div>

风险预算模型：风险预算模型在不同调仓频率下均保持比较稳定的情况。在下图中，可以发现对于风险预算模型而言，回报越高，对应的波动情况也越高。以此次结果而言，以90天作为调仓频率，风险预算模型有最好的预测表现。

<div style="text-align: left;">
    <img src="img/rb_return.png" style="width: 45%; margin-right: 5px;" />
    <img src="img/rb_vol.png" style="width: 45%;" />
</div>

#### 2. 四种模型性能比较：

在四种模型当中，风险预算模型和风险平价模型表现比较稳定。其中风险预算模型表现出更好的回报，以及和平权接近的波动性。而风险平价模型则表现出较差的回报，以及较低波动性。

<div style="text-align: left;">
    <img src="img/rp_rb_return.png" style="width: 45%; margin-right: 5px;" />
    <img src="img/rp_rb_vol.png" style="width: 45%;" />
</div>

均值方差模型和Black-litterman模型则表现得非常不稳定，根据此次回测很难辨别其表现性质，可能存在更多影响因素。

<div style="text-align: left;">
    <img src="img/mvo_bl_return.png" style="width: 45%; margin-right: 5px;" />
    <img src="img/mvo_bl_vol.png" style="width: 45%;" />
</div>

#### 3. 风险预算模型：

在发现风险预算模型的较好表现之后，此实验继续分析了在风险预算模型中，不同风险分配情况对模型的影响。在实验过程中，三种风险分配方法（使用未来情况，使用历史情况，使用garch模型）均表现出正回报以及和平权接近的波动性。其中，使用未来情况 > 使用garch模型 > 使用历史情况。这一结果表明，风险预算越接近实际风险情况，风险预算模型效果将更加有效。与此同时，garch模型表现出一定的根据历史数据预测未来波动情况的能力。

<div style="text-align: left;">
    <img src="img/rb_models_return.png" style="width: 45%; margin-right: 5px;" />
    <img src="img/rb_models_vol.png" style="width: 45%;" />
</div>


对于使用garch模型的风险预算模型，其最佳调仓频率在210天，而对于其他两种风险分配方式，最佳调仓频率均在90天。

<div style="text-align: left;">
    <img src="img/rb_g_return.png" style="width: 45%; margin-right: 5px;" />
    <img src="img/rb_g_vol.png" style="width: 45%;" />
</div>